In [1]:
# !unzip robohearts.zip
#%cd robohearts/

/content/robohearts


In [23]:
import gym
import multiprocessing
from gymhearts.Hearts import *
from gymhearts.Agent.human import Human
from gymhearts.Agent.random_agent import RandomAgent
from gymhearts.Agent.monte_carlo import MonteCarlo
from tqdm import tqdm_notebook

In [2]:
NUM_TESTS = 10
NUM_EPISODES = 1000
TRAINING_ITERS = 10000
MAX_SCORE = 100
playersNameList = ['MonteCarlo', 'Rando', 'Randy', 'Randall']
agent_list = [0, 0, 0, 0]

# Human vs Random
agent_list[0] = MonteCarlo(playersNameList[0], {'print_info' : False})
agent_list[1] = RandomAgent(playersNameList[1], {'print_info' : False})
agent_list[2] = RandomAgent(playersNameList[2], {'print_info' : False})
agent_list[3] = RandomAgent(playersNameList[3], {'print_info' : False})

In [3]:
# TRAIN THE MONTE CARLO AGENT
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
weights = []
for trn_episode in tqdm_notebook(range(TRAINING_ITERS)):
    observation = env.reset()
    history = []
    while True:
        #env.render()

        now_event = observation['event_name']
        IsBroadcast = observation['broadcast']
        action = None
        if IsBroadcast == True:
            for agent in agent_list:
                agent.Do_Action(observation)
        else:
            playName = observation['data']['playerName']
            for agent in agent_list:
                if agent.name == playName:
                    action = agent.Do_Action(observation)

        # update my agent
        if now_event == 'RoundEnd':
            agent_list[0].update_weights(history, -reward['MonteCarlo'])
            history = []
            weights = agent_list[0].weight_vec
        if now_event == 'GameOver':
              break
        if not IsBroadcast and observation['data']['playerName'] == 'MonteCarlo':
            history.append(observation)
        observation, reward, done, info = env.step(action)


In [31]:
# Uncomment this line for pretrained weights
# weights = [-0.46031637, -1.02296217, -1.64597146, 0.50871499, 0.05907032, -0.04527117,
# -2.67590788, -1.78400492, -0.08667306, 0.48108891, 0.66066313, -1.57675411,
# -0.56494518, -0.07736412, -0.3257198, -0.65003209, -0.63740714, 0.44494984,
# -0.1545964, 0.67457139, 2.31472314, 0.8694452, -2.29173301, 0.52783125,
# -0.86950875, -1.77655688, -3.29970913, -0.242993, -1.57548922, -1.34238258,
# 0.36816378, -3.23065985, -0.07919411, -2.1089143, -3.12815169, -0.74580836,
# 0.98398675, -0.75271283, -0.81051661, -0.60567687, -3.42010519, -0.63186969,
# -2.02352157, -0.27534069, -0.28736574, -1.15836776, -3.28679005, -0.33767846,
# -0.41568405, 0.2782292, -1.23761129, -1.80559854]

# EVALUATE THE MONTE CARLO AGENT
    
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
agent_list[0] = MonteCarlo(playersNameList[0], params={'weight_vec' : weights})
mc_wins = [0] * NUM_TESTS
def run_test(num_won):
    # Weird hack to make progress bars render properly
    print(' ', end='', flush=True)
    for i_ep in tqdm_notebook(range(NUM_EPISODES)):
        observation = env.reset()
        while True:
            now_event = observation['event_name']
            IsBroadcast = observation['broadcast']
            action = None
            if IsBroadcast == True:
                for agent in agent_list:
                    agent.Do_Action(observation)
            else:
                playName = observation['data']['playerName']
                for agent in agent_list:
                    if agent.name == playName:
                        action = agent.Do_Action(observation)
            if now_event == 'GameOver':
                num_won += int(observation['data']['Winner'] == 'MonteCarlo')
                break
            observation, reward, done, info = env.step(action)
    return num_won
        
pool = multiprocessing.Pool(processes=NUM_ITERS)
mc_wins = pool.map(run_test, mc_wins)
print(mc_wins)

[284, 289, 308, 302, 303, 303, 306, 314, 309, 317]


In [5]:
# EVALUATE THE RANDOM AGENT
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
playersNameList[0] = 'Randman'
agent_list[0] = RandomAgent(playersNameList[0])
rand_wins = [0] * NUM_TESTS
def run_test(num_won):
    # Weird hack to make progress bars render properly
    print(' ', end='', flush=True)
    for i_ep in tqdm_notebook(range(NUM_EPISODES)):
        observation = env.reset()
        while True:
            now_event = observation['event_name']
            IsBroadcast = observation['broadcast']
            action = None
            if IsBroadcast == True:
                for agent in agent_list:
                    agent.Do_Action(observation)
            else:
                playName = observation['data']['playerName']
                for agent in agent_list:
                    if agent.name == playName:
                        action = agent.Do_Action(observation)
            if now_event == 'GameOver':
                num_won += int(observation['data']['Winner'] == 'Randman')
                break
            observation, reward, done, info = env.step(action)
    return num_won
        
pool = multiprocessing.Pool(processes=NUM_ITERS)
rand_wins = pool.map(run_test, rand_wins)
print(rand_wins)

Finished iter: 999 and Random Agent won 257 games


Finished iter: 999 and Random Agent won 229 games


Finished iter: 999 and Random Agent won 269 games


Finished iter: 999 and Random Agent won 261 games


Finished iter: 999 and Random Agent won 260 games


Finished iter: 999 and Random Agent won 266 games


Finished iter: 999 and Random Agent won 257 games


Finished iter: 999 and Random Agent won 257 games


Finished iter: 999 and Random Agent won 252 games


Finished iter: 999 and Random Agent won 262 games



In [34]:
print(f"Monte Carlo won {sum(mc_wins)/len(mc_wins)} times on average :: {str(mc_wins)}")
print(f"Random won {sum(rand_wins)/len(rand_wins)} times on average :: {str(rand_wins)}")
print(f"\n\nThe Monte Carlo weights are: {str(weights)}")

Monte Carlo won 303.5 times on average :: [284, 289, 308, 302, 303, 303, 306, 314, 309, 317]
Random won 257.0 times on average :: [257, 229, 269, 261, 260, 266, 257, 257, 252, 262]


The Monte Carlo weights are: [-0.46031637, -1.02296217, -1.64597146, 0.50871499, 0.05907032, -0.04527117, -2.67590788, -1.78400492, -0.08667306, 0.48108891, 0.66066313, -1.57675411, -0.56494518, -0.07736412, -0.3257198, -0.65003209, -0.63740714, 0.44494984, -0.1545964, 0.67457139, 2.31472314, 0.8694452, -2.29173301, 0.52783125, -0.86950875, -1.77655688, -3.29970913, -0.242993, -1.57548922, -1.34238258, 0.36816378, -3.23065985, -0.07919411, -2.1089143, -3.12815169, -0.74580836, 0.98398675, -0.75271283, -0.81051661, -0.60567687, -3.42010519, -0.63186969, -2.02352157, -0.27534069, -0.28736574, -1.15836776, -3.28679005, -0.33767846, -0.41568405, 0.2782292, -1.23761129, -1.80559854]
